# Sample covariance

In [1]:
import numpy as np
import cvxpy as cp

from cvx.risk.sample import SampleCovariance
from cvx.risk.random import rand_cov

n = 6

In [2]:
def problem(n):
    weights = cp.Variable(n)
    _riskmodel = SampleCovariance(num=n)

    _problem = cp.Problem(
        cp.Minimize(_riskmodel.estimate_risk(weights)),
        [cp.sum(weights) == 1.0, weights >= 0, _riskmodel.lower <= weights, weights <= _riskmodel.upper],
    )

    return _problem, _riskmodel

In [3]:
%%timeit
n = 50
a = rand_cov(n-2)

_problem, _riskmodel = problem(n)

for i in range(100):
    _riskmodel.update_data(cov=a, lower=np.zeros(48), upper=np.ones(48))
    _problem.solve()

270 ms ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
n = 50
a = rand_cov(n-2)

# here we construct a cvxpy problem in every iteration.
# We do not take advantage of parameters 

for i in range(100):
    _problem, _riskmodel = problem(a.shape[0])
    _riskmodel.update_data(cov=a, lower=np.zeros(48), upper=np.ones(48))
    _problem.solve()

6.87 s ± 1.17 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
